# December 10, 2023

https://adventofcode.com/2023/day/10

In [139]:
from collections import deque

In [236]:
test1_str = f'''-L|F7
7S-7|
L|7||
-L-J|
L|-JF'''
test1 = test1_str.split("\n")

test2_str = f'''7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ'''
test2 = test2_str.split("\n")

In [1]:
fn = "../data/2023/10.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz = [x.strip() for x in text]

In [5]:
test

['-L|F7', '7S-7|', 'L|7||', '-L-J|', 'L|-JF']

In [225]:
def symbol( cur, maze ):
    return maze[cur[0]][cur[1]]

def find_dirs( s ):
    if s == "F":
        return "down", "right"
    if s == "|":
        return "down", "up"
    if s == "7":
        return "down", "left"
    if s == "J":
        return "up", "left"
    if s == "L":
        return "up", "right" 
    if s == "-":
        return "left", "right"
    if s == ".":
        return None
    raise("Unexpected map symbol", s)

def is_from_dir( cur, last, dir ):
    if dir == "down":
        return last[0] == cur[0]+1 and last[1] == cur[1]
    if dir == "up":
        return last[0] == cur[0]-1 and last[1] == cur[1]
    if dir == "left":
        return last[1] == cur[1]-1 and last[0] == cur[0]
    if dir == "right":
        return last[1] == cur[1]+1 and last[0] == cur[0]
    raise("Unexpected dir", dir)

def go_dir( cur, dir ):
    if dir == "down":
        return cur[0]+1, cur[1]
    if dir == "up":
        return cur[0]-1, cur[1]
    if dir == "left":
        return cur[0], cur[1]-1
    if dir == "right":
        return cur[0], cur[1]+1
    raise("Unexpected dir", dir)
 

def find_next_coord( cur, last, maze ):
    s = symbol( cur, maze )
    if s == ".":
        return None
    dir1, dir2 = find_dirs( s )
    if is_from_dir( cur, last, dir1 ):
        return go_dir( cur, dir2 )
    else:
        return go_dir( cur, dir1 )
    raise("WFAS")

def opp_dir( dir ):
    if dir == "down":
        return "up"
    if dir == "up":
        return "down"
    if dir == "left":
        return "right"
    if dir == "right": 
        return "left"
    raise("Unexpected dir", dir)

def next_move( cur, last, maze ):
    next = find_next_coord( cur, last, maze )

    # edge case: starting from a .
    if next is None:
        return None

    #print("looking at:", next)
    if ( next[0] < 0 or next[0] >= len(maze) or
         next[1] < 0 or next[1] >= len(maze[0]) ):
        return None
    
    next_sym = symbol(next, maze)
    #print( next_sym )
    if next_sym == ".":
        return None
    if next_sym == "S":
        return "S"

    next_dir = find_dirs( next_sym )
    if is_from_dir( next, cur, next_dir[0] ):
        return opp_dir( next_dir[0] )
    if is_from_dir( next, cur, next_dir[1] ):
        return opp_dir( next_dir[1] )
    return None


def travel_path( cur, last, maze, verbose=False ):
    path = [last, cur]

    while True:
        if verbose:
            print(f'''@{cur[0]}, {cur[1]}, {symbol(cur, maze)}''', end='')
        to_go = next_move( cur, last, maze )
        # Can't go that way... dead end!
        if to_go is None:
            if verbose:
                print('\tPATH ENDED!')
            return None
        if verbose:
            print("\tGoing: ", to_go)
        # Back to the start!
        if to_go == "S":
            return path

        last = cur
        cur = go_dir( cur, to_go )
        path.append( cur )

def find_start( maze ):
    for row, line in enumerate(maze):
        if 'S' in line:
            for col, char in enumerate(line):
                if char == 'S':
                    return row, col
    raise( BaseException, 'no S found' )

def find_loop( maze, verbose = False ):
    start = find_start( maze )
    if verbose:
        print("Starting at", *start)
    
    beginnings = ( go_dir(start, 'up'),
                   go_dir(start, 'down'),
                   go_dir(start, 'left'),
                   go_dir(start, 'right')
                )
    for b in beginnings:
        if verbose:
            print("\nTry starting at", *b, )

        # Algo doesn't handle the first check!
        bsym = symbol(b, maze)
        if bsym == ".":
            if verbose:
                print("Can't start at", bsym)
            continue

        bdir1, bdir2 = find_dirs( bsym )
        if not is_from_dir(b, start, bdir1) and not is_from_dir(b, start, bdir2 ):
            if verbose:
                print("Can't start at", bsym, "from this direction")
            continue

        loop = travel_path(b, start, maze, verbose=verbose )
        if loop is not None:
            break
    if loop is None:
        raise( BaseException, 'No loop found!' )
    
    return loop

### Part 1

In [ ]:

def part1( maze, verbose=False ):
    loop = find_loop( maze, verbose )
    return int(len(loop)/2)


In [216]:
part1(test1, verbose=True) 

Starting at 1 1

Try starting at 0 1
Can't start at L from this direction

Try starting at 2 1
@2, 1, |	Going:  down
@3, 1, L	Going:  right
@3, 2, -	Going:  right
@3, 3, J	Going:  up
@2, 3, |	Going:  up
@1, 3, 7	Going:  left
@1, 2, -	Going:  S


4

In [217]:
part1(test2, verbose=True) 

Starting at 2 0

Try starting at 1 0
Can't start at .

Try starting at 3 0
@3, 0, |	Going:  down
@4, 0, L	Going:  right
@4, 1, J	Going:  up
@3, 1, F	Going:  right
@3, 2, -	Going:  right
@3, 3, -	Going:  right
@3, 4, J	Going:  up
@2, 4, 7	Going:  left
@2, 3, L	Going:  up
@1, 3, |	Going:  up
@0, 3, 7	Going:  left
@0, 2, F	Going:  down
@1, 2, J	Going:  left
@1, 1, F	Going:  down
@2, 1, J	Going:  S


8

In [218]:
part1( puzz, verbose=False )

7097

### Part 2

Not really sure how to go about this...

In [ ]:
def start_pipe_shape( start, loop ):
    one = loop[0]
    two = loop[-1]

    # left-right case: make sure one is on the left of S
    if one[0] == two[0]:
        if one[1] > two[1]:
            tmp=two; two=one; one=tmp
    # otherwise: make sure one is the higher one
    else:
        if one[0] > two[0]:
            tmp=two; two=one; one=tmp

    # Now we have a priority: up, left, right, down and one has higher priority

    if is_from_dir( start, one, "up" ):
        if is_from_dir( start, two, "down"):
            return "|"
        if is_from_dir( start, two, "left"):
            return "J"
        if is_from_dir( start, two, "right"):
            return "L"
    if is_from_dir( start, one, "left" ):
        if is_from_dir( start, two, "right" ):
            return "-"
        if is_from_dir( start, two, "down" ):
            return "7"
    if is_from_dir( start, one, "right" ):
        if is_from_dir( start, two, "down" ):
            return "F"
    raise( BaseException, "couldn't determine start shape")
            


def zoom_map( maze, verbose=False ):
    start = find_start(maze)
    loop = find_loop(maze, verbose)

    ssym = start_pipe_shape( start, loop )

In [219]:
# Rather than try to figure out discontiguous areas and squeezing through pipes,
# we'll just zoom in and embiggen the space
# This means that the nest interior will be contiguous,
# but we'll have to be careful which spaces we count.

def double( vec ):
    return tuple(2*x for x in vec)

def mean( vec1, vec2 ):
    return tuple( int((x+y)/2) for x,y in zip(vec1, vec2) )

def zoom_loop( loop, verbose=False ):
    big_loop = [ double(loop[0]) ]
    #big_loop = []
    for pos in loop[1:]:
        next = double(pos)
        entre = mean( big_loop[-1], next )
        big_loop += [ entre, next ]

    big_loop.append( mean( big_loop[-1], big_loop[0]) )

    return big_loop

In [233]:
def neighbors( cur, incl_diag=False ):
    up = go_dir(cur, 'up')
    dwn = go_dir(cur, 'down')
    lft = go_dir(cur, 'left')
    rgt = go_dir(cur, 'right')

    if incl_diag:
        return (
            go_dir(up, 'left'), up, go_dir(up, 'right'),
            lft, rgt,
            go_dir(dwn, 'left'), dwn, go_dir(dwn, 'right'),
        )
    else:
        return (up, lft, rgt, dwn)

def encode( cur ):
    return f'''{cur[0]}:{cur[1]}'''

def decode( chr ):
    return [int(x) for x in chr.split(":")]

def on_edge( pos, maze ):
    # note double the maze dimensions since the coords have been doubled from zoom_loop
    return ( pos[0] <= 0 or pos[1] <= 0 or
             pos[0] >= len(maze)*2 or pos[1] >= len(maze[0])*2 )


def map_interior( loop, maze, verbose=False ):
    loop = sorted( loop )
    start = loop[0]
    beginnings = neighbors( start, incl_diag = True )

        
    for b in beginnings:
        if verbose:
            print("\nbeginning at", *b)

        if b in loop: # oops, not interior!
            if verbose:
                print("part of loop. moving on...")
            continue

        if on_edge( b, maze ):
            if verbose:
                print("this is on the exterior. moving on...")
            continue

        chr = encode(b)
        frontier = set( [chr] )
        room = set( [chr] )
        off_the_grid = False

        while len(frontier) > 0:
            cur_str = frontier.pop()
            cur = decode( cur_str )

            nbrs = neighbors(cur)
            if verbose:
                print("Cur:", cur_str)

            for n in nbrs:
                if verbose:
                    print(*n)
                # check neighbors

                if n in loop:
                    # n isn't part of the space
                    continue

                # "edge" case... we've gone off the grid, this is the room OUTSIDE the loop
                if on_edge( n, maze ):
                    off_the_grid = True
                    break
                # get it? "edge" case... because it's on the edge of the map?

                # otherwise, add it to the room if it's not already there
                # also add it to the frontier because we haven't explored it yet
    
                n_str = encode(n)
                if not n_str in room:
                    room.add( n_str )
                    frontier.add( n_str )
        
        if not off_the_grid:
            # we explored everything, this is the interior room!
            return room
        
        # we reached an edge. that was the exterior
        if verbose:
            print("OFF COURSE!")

    return None

def room_size( room ):
    tot = 0
    for r_str in room:
        # if both coords aren't even, it's an expansion space, not one of the original squares
        r = decode(r_str)
        if (r[0] % 2 == 0) and (r[1] % 2 == 0):
            tot += 1

    return tot

                
def part2( maze, verbose = False ):
    loop = find_loop( maze, verbose )
    zl = zoom_loop( loop, verbose )
    room = map_interior( zl, maze, verbose )
    return room_size( room )                    




In [238]:
part2( test1, verbose=True)

Starting at 1 1

Try starting at 0 1
Can't start at L from this direction

Try starting at 2 1
@2, 1, |	Going:  down
@3, 1, L	Going:  right
@3, 2, -	Going:  right
@3, 3, J	Going:  up
@2, 3, |	Going:  up
@1, 3, 7	Going:  left
@1, 2, -	Going:  S

beginning at 1 1
Cur: 1:1
0 1
OFF COURSE!

beginning at 1 2
Cur: 1:2
0 2
OFF COURSE!

beginning at 1 3
Cur: 1:3
0 3
OFF COURSE!

beginning at 2 1
Cur: 2:1
1 1
2 0
Cur: 1:1
0 1
OFF COURSE!

beginning at 2 3
part of loop. moving on...

beginning at 3 1
Cur: 3:1
2 1
3 0
Cur: 2:1
1 1
2 0
Cur: 1:1
0 1
OFF COURSE!

beginning at 3 2
part of loop. moving on...

beginning at 3 3
Cur: 3:3
2 3
3 2
3 4
4 3
Cur: 4:3
3 3
4 2
4 4
5 3
Cur: 3:4
2 4
3 3
3 5
4 4
Cur: 3:5
2 5
3 4
3 6
4 5
Cur: 4:5
3 5
4 4
4 6
5 5
Cur: 4:4
3 4
4 3
4 5
5 4
Cur: 5:4
4 4
5 3
5 5
6 4
Cur: 5:3
4 3
5 2
5 4
6 3
Cur: 5:5
4 5
5 4
5 6
6 5


1

In [239]:
part2(test2, verbose=True )

Starting at 2 0

Try starting at 1 0
Can't start at .

Try starting at 3 0
@3, 0, |	Going:  down
@4, 0, L	Going:  right
@4, 1, J	Going:  up
@3, 1, F	Going:  right
@3, 2, -	Going:  right
@3, 3, -	Going:  right
@3, 4, J	Going:  up
@2, 4, 7	Going:  left
@2, 3, L	Going:  up
@1, 3, |	Going:  up
@0, 3, 7	Going:  left
@0, 2, F	Going:  down
@1, 2, J	Going:  left
@1, 1, F	Going:  down
@2, 1, J	Going:  S

beginning at -1 3
this is on the exterior. moving on...

beginning at -1 4
this is on the exterior. moving on...

beginning at -1 5
this is on the exterior. moving on...

beginning at 0 3
this is on the exterior. moving on...

beginning at 0 5
part of loop. moving on...

beginning at 1 3
Cur: 1:3
0 3
OFF COURSE!

beginning at 1 4
part of loop. moving on...

beginning at 1 5
Cur: 1:5
0 5
1 4
1 6
2 5
Cur: 2:5
1 5
2 4
2 6
3 5
Cur: 3:5
2 5
3 4
3 6
4 5
Cur: 4:5
3 5
4 4
4 6
5 5
Cur: 4:4
3 4
4 3
4 5
5 4
Cur: 5:4
4 4
5 3
5 5
6 4
Cur: 5:3
4 3
5 2
5 4
6 3
Cur: 4:3
3 3
4 2
4 4
5 3
Cur: 3:4
2 4
3 3
3 5
4 4

1

In [240]:
test3_str = f'''FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L'''

test3 = test3_str.split("\n")

In [241]:
part2( test3 )

10

In [244]:
part2( puzz, verbose=False )

355